In [ ]:
import json
with open('response.json')  as f:
    response = json.load(f)

In [ ]:

import pandas as pd
data = pd.read_csv("jobs.csv")
data["start_date"] = pd.to_datetime(data["start_date"], format="%d/%m/%Y")
data["end_date"] = pd.to_datetime(data["end_date"], format="%d/%m/%Y")


In [ ]:
data[data["company"] == "address"]["description"].to_list()[0]

In [ ]:
# Iterate over all type == job

for job in data[data["type"] == "job"].iterrows():
    print(job[1]["company"])
    print(job[1]["description"])
    print("----")

In [ ]:
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import docx

#This is only needed if you're using the builtin style above
def get_or_create_hyperlink_style(d):
    """If this document had no hyperlinks so far, the builtin
       Hyperlink style will likely be missing and we need to add it.
       There's no predefined value, different Word versions
       define it differently.
       This version is how Word 2019 defines it in the
       default theme, excluding a theme reference.
    """
    if "Hyperlink" not in d.styles:
        if "Default Character Font" not in d.styles:
            ds = d.styles.add_style("Default Character Font",
                                    docx.enum.style.WD_STYLE_TYPE.CHARACTER,
                                    True)
            ds.element.set(docx.oxml.shared.qn('w:default'), "1")
            ds.priority = 1
            ds.hidden = True
            ds.unhide_when_used = True
            del ds
        hs = d.styles.add_style("Hyperlink",
                                docx.enum.style.WD_STYLE_TYPE.CHARACTER,
                                True)
        hs.base_style = d.styles["Default Character Font"]
        hs.unhide_when_used = True
        hs.font.color.rgb = docx.shared.RGBColor(0x05, 0x63, 0xC1)
        hs.font.underline = True
        del hs

    return "Hyperlink"
def add_hyperlink(paragraph, text, url):
    # This gets access to the document.xml.rels file and gets a new relation id value
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a new run object (a wrapper over a 'w:r' element)
    new_run = docx.text.run.Run(
        docx.oxml.shared.OxmlElement('w:r'), paragraph)
    new_run.text = text

    # Set the run's style to the builtin hyperlink style, defining it if necessary
    new_run.style = get_or_create_hyperlink_style(part.document)
    # Alternatively, set the run's formatting explicitly
    # new_run.font.color.rgb = docx.shared.RGBColor(0, 0, 255)
    # new_run.font.underline = True

    # Join all the xml elements together
    hyperlink.append(new_run._element)
    paragraph._p.append(hyperlink)
    return hyperlink

import os

# Create the Word document
document = Document()

document.add_heading(f"{data[data["company"] == "name"]["description"].to_list()[0]} - {response["resume_section"]["title"]}" , 0)

document.add_paragraph(f"{data[data["company"] == "address"]["description"].to_list()[0]} • {data[data["company"] == "phone"]["description"].to_list()[0]}")
p = document.add_paragraph(f"{data[data['company'] == 'email']['description'].to_list()[0]} • ")
websites = data[data["company"] == "website"]["description"].to_list()
for i, website in enumerate(websites):
    add_hyperlink(p, website, website)
    if i < len(websites) - 1:
        p.add_run(" • ")

document.add_heading('SKILLS', level=1)
skill:dict
for skill in response["resume_section"].get("skills",[]):
    p = document.add_paragraph()
    p.style = "List Bullet"
    p.add_run(skill["name"]).bold = True
    p.add_run(f" - {skill['description']}")
document.add_heading('EXPERIENCE', level=1)

experience:dict
for experience in response["resume_section"].get("experience",[]): 
    p = document.add_paragraph()
    p.add_run(experience["company"]).bold = True
    p.add_run(f" • {experience['location']} • {experience['position']}")
    
    # Add a tab stop for right alignment
    tab_stop = p.paragraph_format.tab_stops.add_tab_stop(Inches(6.5))  # Adjust width as needed
    tab_stop.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    p.add_run(f"\t({experience['start_date']} - {experience['end_date'] if experience['end_date'] else 'Present'})")
    p = document.add_paragraph()
    p.add_run(experience["description"])


document.add_heading('EDUCATION AND CERTIFICATIONS', level=1)
for education in data[data["type"] == "education"].iterrows():
    p = document.add_paragraph(f"{education[1]["company"]}, {education[1]["location"]} • {education[1]["description"]}")
    tab_stop = p.paragraph_format.tab_stops.add_tab_stop(Inches(6.5))  # Adjust width as needed
    tab_stop.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    p.add_run(f"\t{education[1]["start_date"].strftime('%m/%Y')} - {education[1]["end_date"].strftime('%m/%Y') if education[1]["end_date"] else 'Present'}")



# Save the Word document
docx_path = 'demo.docx'
document.save(docx_path)


# Convert the Word document to PDF
try:
    import comtypes.client

    def convert_to_pdf(docx_path, pdf_path):
        word = comtypes.client.CreateObject('Word.Application')
        doc = word.Documents.Open(os.path.abspath(docx_path))
        # Shrink content to fit on one page
        doc.PageSetup.FitText = True
        doc.SaveAs(os.path.abspath(pdf_path), FileFormat=17)  # 17 is the PDF format
        doc.Close()
        word.Quit()

    pdf_path = 'demo.pdf'
    convert_to_pdf(docx_path, pdf_path)
    print(f"Document successfully saved as {pdf_path}")

except ImportError:
    print("comtypes library is required to convert to PDF. Install it using 'pip install comtypes'")

In [ ]:
import zipfile, re

def getDocxPageCount(docx_fpath):
    docx_object = zipfile.ZipFile(docx_fpath)
    docx_property_file_data = docx_object.read('docProps/app.xml').decode()
    page_count = re.search(r"<Pages>(\d+)</Pages>", docx_property_file_data).group(1)
    return int(page_count)
getDocxPageCount("demo.docx")

In [ ]:
from IPython.display import Image, display

try:for _, job in data[data["type"] == "job"].iterrows():
    # Create a paragraph for the job entry
    p = document.add_paragraph()
    
    # Add the company, location, and role
    run = p.add_run(f"{job['company']}, {job['location']} • {job['role']}")
    run.bold = True  # Make it bold if needed

    # Add a tab stop for right alignment
    tab_stop = p.paragraph_format.tab_stops.add_tab_stop(Inches(6.5))  # Adjust width as needed
    tab_stop.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

    # Add the start-end date aligned to the right
    p.add_run("\t")  # Add a tab character
    p.add_run(f"({job['start_date'].strftime('%m/%Y')}) - {job['end_date'].strftime('%m/%Y') if pd.notnull(job['end_date']) else 'Present'}")

    # Add the job description below
    document.add_paragraph(job["description"])
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
def stream_graph_updates(user_input: str):
    for event in graph.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break